In [ ]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from datetime import timedelta

# Step 1: Load Monthly FSI Data

df = pd.read_csv('FSI.xlsx', parse_dates=['Date'])
df = df.sort_values('Date')

# Step 2: Apply DWT (Daubechies)

wavelet = 'db4'

# Extract FSI values
fsi = df['FSI'].values

# Perform Discrete Wavelet Transform
coeffs = pywt.wavedec(fsi, wavelet)

# Optional: Denoise by zeroing out detail coefficients
# Keep approximation, remove some detail (adjust level)
coeffs[1:] = [np.zeros_like(detail) for detail in coeffs[1:]]

# Reconstruct the smoothed signal
fsi_denoised = pywt.waverec(coeffs, wavelet)

# Truncate to original length if needed
fsi_denoised = fsi_denoised[:len(fsi)]

# Add denoised values back to dataframe
df['FSI_denoised'] = fsi_denoised


# Step 3: Convert Monthly to Weekly (Interpolate)

# Create a new weekly date range from min to max
weekly_dates = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='W')

# Create a new dataframe for interpolation
df_weekly = pd.DataFrame({'Date': weekly_dates})

# Merge and interpolate
df_interpolated = pd.merge(df_weekly, df[['Date', 'FSI_denoised']], on='Date', how='left')
df_interpolated['FSI_weekly'] = df_interpolated['FSI_denoised'].interpolate(method='time')

# Drop the intermediate column
df_interpolated.drop(columns=['FSI_denoised'], inplace=True)


# Step 4: Save or Plot

# Save to CSV
df_interpolated.to_csv('weekly_fsi_dwt_interpolated.csv', index=False)

# Optional: Plot result
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['FSI'], label='Original Monthly FSI', marker='o')
plt.plot(df['Date'], df['FSI_denoised'], label='Denoised FSI', linestyle='--')
plt.plot(df_interpolated['Date'], df_interpolated['FSI_weekly'], label='Weekly Interpolated', alpha=0.7)
plt.legend()
plt.title('FSI - Monthly to Weekly using DWT and Interpolation')
plt.xlabel('Date')
plt.ylabel('FSI Value')
plt.grid(True)
plt.tight_layout()
plt.show()
